# Test con random forest del sklearn

# Test de los datos de entrenamiento

In [1]:
import pandas as pd
import numpy as np
from numpy import linalg as LA
from sklearn.ensemble import RandomForestClassifier

## Lectura de los datos

In [2]:
df_postulaciones=pd.read_csv('training.csv')

In [3]:
len(df_postulaciones)

9082431

Trabajaremos con un set reducido por causa de las dimensiones del set original

In [4]:
df_postulaciones=df_postulaciones.sample(4000000)

## Distancias VDM entre Categoricals

#### Se determinarán las distancias entre los atributos categóricos nombre_area, tipo_de_trabajo, nivel_laboral y sexo (VDM, value distance measure apunte cap.11.1.11)

In [5]:
#Series con las listas de nombre_area, tipo_de_trabajo, y nivel_laboral respectivamente
#Son 188 nombre_area, 5 nivel_laboral, 10 tipo_de_trabajo
nombre_area=df_postulaciones.nombre_area.dropna().unique()  
tipo_de_trabajo=df_postulaciones.tipo_de_trabajo.dropna().unique() 
nivel_laboral=df_postulaciones.nivel_laboral.dropna().unique() 

In [6]:
len(nombre_area)

188

Nivel_laboral y tipo_de_trabajo y sexo ordenados:

nivel_laboral: Senior/Semi-Senior, Junior, Otro, Jefe/Supervisor/Responsable, Gerencia/Alta Gerencia/Dirección

tipo_de_trabajo: Full-time, Part-time, Pasantia, Por Horas, Temporario, Por Contrato, Teletrabajo, Fines de Semana, Primer empleo. voluntario

sexo: FEM, MASC

Las probabilidades de cada categorical se calcularán para cada clase, siendo estas dadas por la columna post: 1 ó 0

In [9]:
#Matrices con la probabilidad de cada Categorical (nombre_area, sexo, tipo_de_trabajo y nivel_laboral) por cada clase (post) 
#como columna, siendo que la primera columna de cada matriz es la probabilidad de postular (post=1) y la segunda de no postular
#(post=0)

#inicialización de matrices
prob_nombre_area=np.zeros((len(nombre_area),2))
prob_nivel_laboral=np.zeros((len(nivel_laboral),2))
prob_tipo_de_trabajo=np.zeros((len(tipo_de_trabajo),2))
prob_sexo=np.zeros((2,2))

total_post=df_postulaciones.post.value_counts()

prob_nombre_area[:,0]=pd.Series(df_postulaciones[df_postulaciones.post==1].nombre_area.value_counts(),index=nombre_area).fillna(value=0)/total_post[1]
prob_nombre_area[:,1]=pd.Series(df_postulaciones[df_postulaciones.post==0].nombre_area.value_counts(),index=nombre_area).fillna(value=0)/total_post[0]

prob_nivel_laboral[:,0]=pd.Series(df_postulaciones[df_postulaciones.post==1].nivel_laboral.value_counts(),index=nivel_laboral).fillna(value=0)/total_post[1]
prob_nivel_laboral[:,1]=pd.Series(df_postulaciones[df_postulaciones.post==0].nivel_laboral.value_counts(),index=nivel_laboral).fillna(value=0)/total_post[0]

prob_tipo_de_trabajo[:,0]=pd.Series(df_postulaciones[df_postulaciones.post==1].tipo_de_trabajo.value_counts(),index=tipo_de_trabajo).fillna(value=0)/total_post[1]
prob_tipo_de_trabajo[:,1]=pd.Series(df_postulaciones[df_postulaciones.post==0].tipo_de_trabajo.value_counts(),index=tipo_de_trabajo).fillna(value=0)/total_post[0]

prob_sexo[:,0]=pd.Series(df_postulaciones[df_postulaciones.post==1].sexo.value_counts(),index=['FEM','MASC'])/total_post[1]
prob_sexo[:,1]=pd.Series(df_postulaciones[df_postulaciones.post==0].sexo.value_counts(),index=['FEM','MASC'])/total_post[0]

In [10]:
#probabilidades de cada nivel_laboral para cada clase (post) (la suma de cada columna debe dar 1)

pd.DataFrame({'post=1':prob_nivel_laboral[:,0],'post=0':prob_nivel_laboral[:,1]},index=nivel_laboral)

,post=0,post=1
Senior / Semi-Senior,0.671958,0.637698
Jefe / Supervisor / Responsable,0.060096,0.030234
Junior,0.164106,0.220783
Otro,0.077758,0.081141
Gerencia / Alta Gerencia / Dirección,0.012833,0.006330


In [11]:
#probabilidades de cada sexo para cada clase (post) (la suma de cada columna debe dar 1)

pd.DataFrame({'post=1':prob_sexo[:,0],'post=0':prob_sexo[:,1]},index=['FEM','MASC'])

,post=0,post=1
FEM,0.498299,0.516129
MASC,0.450234,0.482142


In [12]:
#probabilidades de cada tipo_de_trabajo para cada clase (post) (la suma de cada columna debe dar 1)

pd.DataFrame({'post=1':prob_tipo_de_trabajo[:,0],'post=0':prob_tipo_de_trabajo[:,1]},index=tipo_de_trabajo)

,post=0,post=1
Full-time,0.903190,0.847327
Por Contrato,0.003452,0.001404
Por Horas,0.005012,0.003476
Part-time,0.068708,0.137403
Pasantia,0.004633,0.004728
Temporario,0.003814,0.002823
Teletrabajo,0.009832,0.001009
Primer empleo,0.000233,0.000334
Fines de Semana,0.001098,0.001454
Voluntario,0.000029,0.000043


In [13]:
#"distancia" entre sexos
abs(prob_sexo[0,:]-prob_sexo[1,:]).sum()

0.0820511757652907

In [14]:
#Matrices cuadradas de distancias VDM entre cada cada categorical. Estas matrices tienen 0 en la diagonal principal (porque la 
#distancia entre cada Categorical con ella misma es cero) y son simétricas (porque al distancia entre A y B es la misma que 
#entre B y A)
VDM_nombre_area=np.zeros((len(nombre_area),len(nombre_area)))
VDM_nivel_laboral=np.zeros((len(nivel_laboral),len(nivel_laboral))) 
VDM_tipo_de_trabajo=np.zeros((len(tipo_de_trabajo),len(tipo_de_trabajo)))  
VDM_sexo=np.zeros((2,2))

for i in range(0,len(nombre_area)):
    for j in range(0,len(nombre_area)):
        VDM_nombre_area[i][j]=abs(prob_nombre_area[i][:]-prob_nombre_area[j][:]).sum()
for i in range(0,len(nivel_laboral)):
    for j in range(0,len(nivel_laboral)):
        VDM_nivel_laboral[i][j]=abs(prob_nivel_laboral[i][:]-prob_nivel_laboral[j][:]).sum()
for i in range(0,len(tipo_de_trabajo)):
    for j in range(0,len(tipo_de_trabajo)):
        VDM_tipo_de_trabajo[i][j]=abs(prob_tipo_de_trabajo[i][:]-prob_tipo_de_trabajo[j][:]).sum()
VDM_sexo[0][1]=abs(prob_sexo[0][:]-prob_sexo[1][:]).sum()
VDM_sexo[1][0]=VDM_sexo[0][1]

In [15]:
#"distancias" entre los tipos de trabajo
VDM_tipo_de_trabajo

array([[0.00000000e+00, 1.74566173e+00, 1.74202878e+00, 1.54440612e+00,
        1.74115603e+00, 1.74388031e+00, 1.73967653e+00, 1.74994983e+00,
        1.74796589e+00, 1.75044537e+00],
       [1.74566173e+00, 0.00000000e+00, 3.63294845e-03, 2.01255608e-01,
        4.50570039e-03, 1.78142019e-03, 6.77431719e-03, 4.28809457e-03,
        2.40420226e-03, 4.78363616e-03],
       [1.74202878e+00, 3.63294845e-03, 0.00000000e+00, 1.97622659e-01,
        1.62972384e-03, 1.85152826e-03, 7.28702746e-03, 7.92104302e-03,
        5.93711141e-03, 8.41658461e-03],
       [1.54440612e+00, 2.01255608e-01, 1.97622659e-01, 0.00000000e+00,
        1.96749907e-01, 1.99474188e-01, 1.95270407e-01, 2.05543702e-01,
        2.03559771e-01, 2.06039244e-01],
       [1.74115603e+00, 4.50570039e-03, 1.62972384e-03, 1.96749907e-01,
        0.00000000e+00, 2.72428020e-03, 8.91675131e-03, 8.79379496e-03,
        6.80986335e-03, 9.28933655e-03],
       [1.74388031e+00, 1.78142019e-03, 1.85152826e-03, 1.99474188e-01,
   

In [16]:
#"distancias" entre niveles laborales
VDM_nivel_laboral

array([[0.        , 1.21932633, 0.92476625, 1.15075659, 1.29049243],
       [1.21932633, 0.        , 0.29456008, 0.06856974, 0.0711661 ],
       [0.92476625, 0.29456008, 0.        , 0.22599034, 0.36572618],
       [1.15075659, 0.06856974, 0.22599034, 0.        , 0.13973584],
       [1.29049243, 0.0711661 , 0.36572618, 0.13973584, 0.        ]])

## Multidimensional scaling

A partir de las matrices de distancia se utilizará multidimensional scaling para determinar las "coordenadas" de cada Categorical tanto en nombre_area, tipo_de_trabajo, como en nivel_laboral. En el caso del sexo no se calculan las coordenadas porque al ser sólo dos puntos estos están sobre una recta (una dimensión). Recuérdese que estas coordenadas pueden ser rotadas y desplazadas que no cambia la matriz.

In [17]:
#Función que devuelve los autovalores  autovectores de la matriz de distancias al cuadrado centrada. 
#los autovalores son devueltos en orden ASCENDENTE
#La matriz de distancia debe ser pasada como parámetro
def autovals(mat_dist):
    n=len(mat_dist)
    J=np.identity(n)-np.ones((n,n))/n     #matriz de centralización (para que el promedio de las líneas y columnas =0)
    mat_dist_2=np.square(mat_dist)        #matriz de distancias con el cuadrado de cada componente
    B=-np.dot(np.dot(J,mat_dist_2),J)/2   #matriz de distancias al cuadrado centralizada
    lamb, v=LA.eigh(B)                    #autovalores y autovectores de la matriz
    return lamb, v

In [18]:
#Función que devuelve las coordenadas de cada Categorical
#la matriz de distancias debe ser pasada como parámetro
#La dimensión de las coordenadas estará dada por la cantidad de autovalores diferentes de cero (con precisión del 95%)
def mdscaling(mat_dist):
    lamb_,v_=autovals(mat_dist)      #calculo autovalores y autovetores de la matriz
    lamb=np.zeros(len(lamb_))
    v=np.zeros((len(lamb_),len(lamb_)))
    for q in range(0,len(lamb_)):     #los ordeno de forma decreciente
        lamb[q]=lamb_[len(lamb_)-q-1]
        v[:,q]=v_[:,len(lamb_)-q-1]
     #print(lamb)
    sum_autovals_total=np.dot(lamb,np.ones((len(lamb),1)))    #suma de todos los autovalores
    q=0  #len(lamb)
    sum_autovals=lamb[q]                                      #suma parcial de autovalores
    while (sum_autovals/sum_autovals_total<0.95):             #voy sumando autovalores hasta llegar al 95% del total
        q+=1
        sum_autovals+=lamb[q]
    #q=0 #provisorio para forzar las coordenadas a una sola dimensión
    X=np.dot(v[:,0:q+1],np.sqrt(np.diag(lamb[0:q+1])))        #devuelve la matriz de autovectores por la raíz cuadrada de los 
                                                              #autovalores <>0 correspondientes
    return X

In [19]:
#Coordinadas de cada variable categorical
X_nivel_laboral=mdscaling(VDM_nivel_laboral)
X_tipo_de_trabajo=mdscaling(VDM_tipo_de_trabajo)
X_nombre_area=mdscaling(VDM_nombre_area)
X_sexo=[0,abs(prob_sexo[0,:]-prob_sexo[1,:]).sum()]

In [20]:
#desloco para ter todos os pontos positivos. Este punto es válido sólo cuando las coordenadas son unidimensionales 
#(lo que generalmente pasa)
X_nivel_laboral-=min(X_nivel_laboral)
X_nombre_area-=min(X_nombre_area)
X_tipo_de_trabajo-=min(X_tipo_de_trabajo)

In [21]:
#coordenada de cada tipo_de_trabajo
pd.Series(X_tipo_de_trabajo[:,0],index=tipo_de_trabajo)

Full-time          1.750446
Por Contrato       0.004784
Por Horas          0.008418
Part-time          0.206039
Pasantia           0.009292
Temporario         0.006567
Teletrabajo        0.010776
Primer empleo      0.000496
Fines de Semana    0.002480
Voluntario         0.000000
dtype: float64

In [22]:
#coordenada de cada nivel_laboral
pd.Series(X_nivel_laboral[:,0],index=nivel_laboral)

Senior / Semi-Senior                    1.290492
Jefe / Supervisor / Responsable         0.071166
Junior                                  0.365726
Otro                                    0.139736
Gerencia / Alta Gerencia / Dirección    0.000000
dtype: float64

In [23]:
#muestra de 10 coordenadas de nombre_area
pd.Series(X_nombre_area[:,0],index=nombre_area).sample(10)

Independientes                   0.000281
Odontología                      0.000047
Estética y Cuidado Personal      0.001340
Educación especial               0.000067
Clínica Médica                   0.000103
Telecomunicaciones               0.007343
Otras áreas técnicas en salud    0.000742
Asistente                        0.007422
Trabajo Social                   0.000110
Seguridad e Higiene              0.003456
dtype: float64

In [24]:
#substituyo las columnas categóricas por sus coordenadas y elimino las columnas que no voy a usar
del df_postulaciones['titulo']
del df_postulaciones['denominacion_empresa']
df_postulaciones.tipo_de_trabajo=list(pd.Series(X_tipo_de_trabajo[:,0],index=tipo_de_trabajo)[df_postulaciones.tipo_de_trabajo.fillna(value=-1)])
df_postulaciones.nivel_laboral=list(pd.Series(X_nivel_laboral[:,0],index=nivel_laboral)[df_postulaciones.nivel_laboral.fillna(value=-1)])
df_postulaciones.nombre_area=list(pd.Series(X_nombre_area[:,0],index=nombre_area)[df_postulaciones.nombre_area.fillna(value=-1)])
df_postulaciones.sexo=list(pd.Series(X_sexo,index=['FEM','MASC'])[df_postulaciones.sexo.fillna(value=-1)])

In [25]:
#sólo para que no joda que a veces coloca el autovalor como lambda+0j
df_postulaciones.tipo_de_trabajo=abs(df_postulaciones.tipo_de_trabajo)
df_postulaciones.nivel_laboral=abs(df_postulaciones.nivel_laboral)
df_postulaciones.nombre_area=abs(df_postulaciones.nombre_area)

In [26]:
#la edad de los no declarantes lo completo por la media
df_postulaciones.edad=df_postulaciones.edad.fillna(value=df_postulaciones.edad.mean())

In [27]:
#la educación no declarada la considero la peor=0
df_postulaciones.valor_educ=df_postulaciones.valor_educ.fillna(value=0)

In [28]:
#y el sexo un promedio entre hombre y mujer
df_postulaciones.sexo=df_postulaciones.sexo.fillna(value=VDM_sexo[0][1]/2)

In [51]:
#hay algunos avisos son nivel laboral declarado, serán substituidos por la mediana, 0.139736 de 'Otro'
df_postulaciones.nivel_laboral=df_postulaciones.nivel_laboral.fillna(value=X_nivel_laboral[3][0])

In [52]:
df_postulaciones.sample(10)

,Unnamed: 0,idaviso,tipo_de_trabajo,nivel_laboral,nombre_area,idpostulante,valor_educ,sexo,edad,post,vistas
669690,669690,1112286523,1.750446,1.290492,0.016965,638eJM,11.0,0.000000,32.143575,1,0.0
428113,428113,1112266774,1.750446,0.139736,0.161871,Oqb6EM3,11.0,0.082051,28.546315,1,0.0
5860148,5860148,1112418673,0.206039,0.365726,0.055635,vVjz1pX,1.0,0.000000,18.283301,1,1.0
5913535,5913535,1112423597,1.750446,1.290492,0.140962,vV3461P,11.0,0.082051,28.431246,1,0.0
1609157,1609157,1111444020,1.750446,1.290492,0.012038,9696v39,10.0,0.082051,24.217548,1,0.0
3588408,3588408,1112439298,0.206039,1.290492,0.161871,akDJwbW,7.0,0.000000,25.650424,1,4.0
2364824,2364824,1112291332,1.750446,1.290492,0.142433,BmB64bL,2.0,0.082051,21.179192,1,0.0
6138613,6138613,1112511883,1.750446,1.290492,0.048797,rm09z5e,10.0,0.082051,22.398370,0,0.0
9034352,9034352,1112197469,1.750446,1.290492,0.023143,GNderz9,11.0,0.082051,37.433986,0,0.0
6170016,6170016,1111796565,1.750446,1.290492,0.263633,eXlbE,0.0,0.082051,51.735356,0,0.0


In [53]:
#Preparado de la matriz de test y del vector de postulaciones
X=np.array(df_postulaciones.loc[:,['tipo_de_trabajo','nivel_laboral','nombre_area','valor_educ','sexo','edad']])
t=np.array(df_postulaciones.post)

In [54]:
#Voy a usar las 3/4 parte de los datos para test y la otra 1/4 parte para validación
medida=np.floor((len(df_postulaciones)*3/4)).astype('int')
Xtest=X[0:medida,:]
ttest=t[0:medida]
Xval=X[medida:len(df_postulaciones),:]
tval=t[medida:len(df_postulaciones)]

In [55]:
np.shape(Xtest)

(3000000, 6)

# Ejecución de la floresta aleatoria

In [56]:
clf = RandomForestClassifier(n_estimators=10,criterion='entropy',max_features=3,n_jobs=-1,max_depth=None,min_samples_split=2)
clf = clf.fit(Xtest, ttest)

In [57]:
#verifico la cantidad de aciertos con el test que falta
test=clf.predict(Xval)


In [58]:
#cantidad de errores (resultado diferente das postulaciones)
abs(tval-test).sum()

248976

## Aplicación al set de test

In [59]:
df_test=pd.read_csv('testing.csv')

In [60]:
df_test

,Unnamed: 0,id,idaviso,titulo,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,idpostulante,valor_educ,sexo,edad,vistas
0,0,0,739260,Asistente Comercial,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,6M9ZQR,20.0,FEM,42.277822,0.0
1,1,1,739260,Asistente Comercial,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,6v1xdL,8.0,MASC,31.025767,0.0
2,2,2,739260,Asistente Comercial,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,ezRKm9,11.0,FEM,36.384671,0.0
3,3,3,758580,Oracle Financials Sr. Application Developer,Full-time,Otro,Tecnologia / Sistemas,TeleTech,1Q35ej,14.0,MASC,68.557274,0.0
4,4,4,758580,Oracle Financials Sr. Application Developer,Full-time,Otro,Tecnologia / Sistemas,TeleTech,EAN4J6,10.0,FEM,32.110698,0.0
5,5,5,758580,Oracle Financials Sr. Application Developer,Full-time,Otro,Tecnologia / Sistemas,TeleTech,8R6pzR,9.0,MASC,28.102479,0.0
6,6,6,776420,Consultores Senior y SemiSr de Procesos,Full-time,Otro,Consultoria,Paradigma Consultores,aZJ2XN,11.0,MASC,29.447685,0.0
7,7,7,776420,Consultores Senior y SemiSr de Procesos,Full-time,Otro,Consultoria,Paradigma Consultores,Nmpo3J,8.0,MASC,54.409329,0.0
8,8,8,776420,Consultores Senior y SemiSr de Procesos,Full-time,Otro,Consultoria,Paradigma Consultores,eVqWar,11.0,FEM,28.239466,0.0
9,9,9,820850,Administrativo de RRHH,Por Contrato,Otro,Recursos Humanos,Luis y Miguel Zanniello SA,6ZBD33,14.0,MASC,37.801109,0.0


In [61]:
#substituyo las columnas categóricas por sus coordenadas y elimino las columnas que no voy a usar en esta primera fase
del df_test['titulo']
del df_test['denominacion_empresa']
df_test.tipo_de_trabajo=list(pd.Series(X_tipo_de_trabajo[:,0],index=tipo_de_trabajo)[df_test.tipo_de_trabajo.fillna(value=-1)])
df_test.nivel_laboral=list(pd.Series(X_nivel_laboral[:,0],index=nivel_laboral)[df_test.nivel_laboral.fillna(value=-1)])
df_test.nombre_area=list(pd.Series(X_nombre_area[:,0],index=nombre_area)[df_test.nombre_area.fillna(value=-1)])
df_test.sexo=list(pd.Series(X_sexo,index=['FEM','MASC'])[df_test.sexo.fillna(value=-1)])

In [62]:
#sólo para que no joda que a veces coloca el autovalor como lambda+0j
df_test.tipo_de_trabajo=abs(df_test.tipo_de_trabajo)
df_test.nivel_laboral=abs(df_test.nivel_laboral)
df_test.nombre_area=abs(df_test.nombre_area)

In [63]:
df_test.head(10)

,Unnamed: 0,id,idaviso,tipo_de_trabajo,nivel_laboral,nombre_area,idpostulante,valor_educ,sexo,edad,vistas
0,0,0,739260,1.750446,0.071166,0.140962,6M9ZQR,20.0,0.000000,42.277822,0.0
1,1,1,739260,1.750446,0.071166,0.140962,6v1xdL,8.0,0.082051,31.025767,0.0
2,2,2,739260,1.750446,0.071166,0.140962,ezRKm9,11.0,0.000000,36.384671,0.0
3,3,3,758580,1.750446,0.139736,0.033569,1Q35ej,14.0,0.082051,68.557274,0.0
4,4,4,758580,1.750446,0.139736,0.033569,EAN4J6,10.0,0.000000,32.110698,0.0
5,5,5,758580,1.750446,0.139736,0.033569,8R6pzR,9.0,0.082051,28.102479,0.0
6,6,6,776420,1.750446,0.139736,0.002881,aZJ2XN,11.0,0.082051,29.447685,0.0
7,7,7,776420,1.750446,0.139736,0.002881,Nmpo3J,8.0,0.082051,54.409329,0.0
8,8,8,776420,1.750446,0.139736,0.002881,eVqWar,11.0,0.000000,28.239466,0.0
9,9,9,820850,0.004784,0.139736,0.031152,6ZBD33,14.0,0.082051,37.801109,0.0


In [67]:
#substituyo los Nan como en el set de entrenamiento
df_test.edad=df_test.edad.fillna(value=df_test.edad.mean())
df_test.valor_educ=df_test.valor_educ.fillna(value=0)
df_test.sexo=df_test.sexo.fillna(value=VDM_sexo[0][1]/2)
df_test.nivel_laboral=df_test.nivel_laboral.fillna(value=X_nivel_laboral[3][0])

In [68]:
#matriz con valores numéricos a ser testada con las coordenadas del hiperplano
Xtest=np.array(df_test.loc[:,['tipo_de_trabajo','nivel_laboral','nombre_area','valor_educ','sexo','edad']])

In [69]:
resultdectree=clf.predict(Xtest)

In [70]:
#número de postulaciones efectivas en el set de prueba
resultdectree.sum()

62506

In [71]:
#preparado del archivo csv final
df_result_dectree=pd.DataFrame({'sepostulo':resultdectree},index=df_test.id)

In [73]:
df_result_dectree.sample(5)

,sepostulo
id,
53893,1
67300,0
36240,1
9175,1
58703,0


In [74]:
df_result_dectree.to_csv('result6.csv')